# 💾 Chapter 4.1: 3D Data Representation

In this notebook, we explore the wild zoo of 3D file formats. Just like images have JPEG and PNG, 3D data comes in many flavors tailored for specific use cases.

**We will cover:**
1.  **ASCII Formats** (`.txt`, `.xyz`): Simple, human-readable, but heavy.
2.  **LiDAR Formats** (`.las`, `.laz`): The industry standard for point clouds.
3.  **Mesh Formats** (`.ply`, `.obj`): Storing surfaces (vertices + faces).
4.  **Voxel Grids**: 3D pixels (volumetric data).
5.  **Bonus**: Constructive Solid Geometry (CSG).

---

**🦊 Expert Tip**
> When working with massive datasets (100M+ points), avoid ASCII formats! Binary formats like `.las` or `.ply` are orders of magnitude faster to read and write.

In [ ]:
import numpy as np
import pandas as pd
import open3d as o3d
import laspy
import matplotlib.pyplot as plt

# Check versions
print(f"Open3D version: {o3d.__version__}")
print(f"Laspy version: {laspy.__version__}")

## 1. ASCII Formats (.txt, .xyz)

These files store one point per line. They are easy to parse but take up a lot of disk space.
We will load `bike_florent.txt`.

In [ ]:
file_txt = "../DATA/bike_florent.txt"

# Method 1: Numpy loadtxt (Basic)
# Note: We skip the header row. Delimiter is ';'
try:
    pcd_np = np.loadtxt(file_txt, delimiter=';', skiprows=1)
    print(f"Loaded {pcd_np.shape[0]} points (Numpy).")
    print("Sample (First 5):\n", pcd_np[:5])
except Exception as e:
    print(f"Error loading txt: {e}")

In [ ]:
# Method 2: Pandas read_csv (Faster for large text files)
df = pd.read_csv(file_txt, delimiter=';')
print(f"Loaded {len(df)} rows (Pandas).")
df.head()

## 2. Visualization with Open3D

To visualize the Numpy/Pandas data, we need to convert it into an Open3D Geometry.

In [ ]:
# Create Open3D objects
pcd = o3d.geometry.PointCloud()

# Assign XYZ coordinates [:, 0:3] columns
pcd.points = o3d.utility.Vector3dVector(pcd_np[:, 0:3])

# Assign Colors [:, 3:6] columns -> Normalize to 0-1 range
if pcd_np.shape[1] >= 6:
    pcd.colors = o3d.utility.Vector3dVector(pcd_np[:, 3:6] / 255.0)

print("Visualizing ASCII Point Cloud...")
o3d.visualization.draw_geometries([pcd], window_name="Bike Point Cloud")

## 3. LiDAR Formats (.las, .laz)

The `.las` format is binary and contains rich metadata (intensity, return number, classification). We use `laspy` to handle it.

In [ ]:
file_las = "../DATA/ITC_BUILDING.las"

# 1. Read the LAS file
las = laspy.read(file_las)

# 2. Inspect Metadata
print(f"Point Count: {las.header.point_count}")
print(f"Scale Factors: {las.header.scales}")
print(f"Offset: {las.header.offsets}")

# 3. Extract Coordinates
print(f"X range: {las.x.min()} - {las.x.max()}")
print(f"Y range: {las.y.min()} - {las.y.max()}")
print(f"Z range: {las.z.min()} - {las.z.max()}")

### Visualizing LAS
We convert the LAS data to Open3D format for viewing.

In [ ]:
# Determine colors (if available)
# LAS colors are often 16-bit (0-65535). We check max value to decide scaling.
if las.red.max() > 255:
    las_colors = np.vstack((las.red, las.green, las.blue)).transpose() / 65535.0
else:
    las_colors = np.vstack((las.red, las.green, las.blue)).transpose() / 255.0

pcd_las = o3d.geometry.PointCloud()
pcd_las.points = o3d.utility.Vector3dVector(np.vstack((las.x, las.y, las.z)).transpose())
pcd_las.colors = o3d.utility.Vector3dVector(las_colors)

print("Visualizing LAS Point Cloud...")
o3d.visualization.draw_geometries([pcd_las], window_name="LiDAR Scan")

## 4. Mesh Formats (.ply, .obj)

Meshes define surfaces. We load a `.ply` mesh which contains triangles.

In [ ]:
file_ply = "../DATA/mesh_terrain.ply"

mesh = o3d.io.read_triangle_mesh(file_ply)
mesh.compute_vertex_normals()

print(f"Mesh loaded: {len(mesh.vertices)} vertices, {len(mesh.triangles)} triangles.")
o3d.visualization.draw_geometries([mesh], window_name="Terrain Mesh", mesh_show_back_face=True)

## 5. Voxel Grids

Voxels are cubes in 3D space. They are useful for downsampling and volumetric analysis.

In [ ]:
file_vox = "../DATA/voxels.ply"

# Depending on how the PLY was saved, it might be a point cloud or a voxel grid object
# Let's try loading as VoxelGrid first, if fails, load as PCD and voxelize
try:
    vox_grid = o3d.io.read_voxel_grid(file_vox)
    print("Loaded VoxelGrid directly.")
except:
    print("Could not load as VoxelGrid directly. Loading as PointCloud...")
    pcd_vox = o3d.io.read_point_cloud(file_vox)
    # Create voxel grid from point cloud
    vox_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd_vox, voxel_size=0.5)
    print("Converted to VoxelGrid.")

o3d.visualization.draw_geometries([vox_grid], window_name="Voxel Structure")

## 6. Constructive Solid Geometry (CSG)

We can create complex shapes by combining simple ones (Union, Intersection, Difference).
Here we interact with meshes programmatically.

In [ ]:
# 1. Create two shapes
cube = o3d.geometry.TriangleMesh.create_box(width=2, height=2, depth=2)
cube.compute_vertex_normals()
cube.paint_uniform_color([0.9, 0.1, 0.1])  # Red

sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.3)
sphere.compute_vertex_normals()
sphere.translate((1, 1, 1))
sphere.paint_uniform_color([0.1, 0.9, 0.1])  # Green

# 2. Visualize them together
print("Showing overlapping Cube and Sphere...")
o3d.visualization.draw_geometries([cube, sphere])

# Note: Open3D standard library does not always have robust boolean CSG built-in without 'Manifold' support.
# However, we can simply visualize their intersection by rendering both.